# Capítulo 6 - Algoritmos com Modelo


Você pode rodar este notebook localmente ou no Colab. Para abrir diretamente no Colab, basta clicar no link abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap06/cap06-main-1.ipynb)

## Configurações Iniciais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg freeglut3-dev xvfb

    !pip install gym==0.23.1
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [ ]:
if IN_COLAB:
    # Set up fake display; otherwise rendering will fail
    import os
    os.system("Xvfb :1 -screen 0 1024x768x24 &")
    os.environ['DISPLAY'] = ':1'

    from util.notebook import display_videos_from_path

In [ ]:
import gym
import numpy as np
import optuna

from util.experiments import repeated_exec, repeated_exec_greedy_Q
from util.plot import plot_result, plot_multiple_results

In [ ]:
VERBOSE = False

## 1 - Q-Learning

Segue o mesmo **Q-learning** que vimos antes (ainda com o critério de parada baseado na quantidade de episódios).

In [ ]:
# Esta é a política. Neste caso, escolhe uma ação com base nos valores
# da tabela Q, usando uma estratégia epsilon-greedy.
def epsilon_greedy(Q, state, num_actions, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, num_actions)
    else:
        return np.argmax(Q[state])

In [ ]:
def run_qlearning(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, render=False):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n
    
    # inicializa a tabela Q com valores aleatórios de -1.0 a 0.0
    # usar o estado como índice das linhas e a ação como índice das colunas
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []
    
    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        
        state = env.reset()
    
        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:
            
            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            if done: 
                # para estados terminais
                V_next_state = 0
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table
            # delta = (estimativa usando a nova recompensa) - estimativa antiga
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta
            
            sum_rewards += reward
            state = next_state

        sum_rewards_per_ep.append(sum_rewards)

        # a cada 100 episódios, imprime informação sobre o progresso 
        if VERBOSE and ((i+1) % 100 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Episode {i+1} Average Reward (last 100): {avg_reward:.3f}")

    return sum_rewards_per_ep, Q

In [ ]:
EPISODES = 500
LR = 0.3
GAMMA = 0.90
EPSILON = 0.1

rmax = 10.0
env = gym.make("Taxi-v3")

rewards, qtable = run_qlearning(env, EPISODES, LR, GAMMA, EPSILON)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards, rmax, cumulative=False, window=10)

## 2 - Dyna-Q ...

Um algoritmo com modelo.

In [ ]:
import random as rand
from math import sqrt

In [ ]:
def planning(model, planning_steps, Q, lr, gamma):
    all_s_a = list(model.keys())
    if len(all_s_a) < planning_steps:
        samples = rand.choices(all_s_a, k=planning_steps)
    else:
        samples = rand.sample(all_s_a, k=planning_steps)
    
    for s, a in samples:
        r, next_s, is_terminal = model[(s,a)]
        if is_terminal:
            V_next_s = 0
        else:
            V_next_s = np.max(Q[next_s])        
        delta = (r + gamma * V_next_s) - Q[s,a]
        Q[s,a] = Q[s,a] + lr * delta

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

   # inicializa a tabela Q
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    model = dict({})  # keys are pairs (state, action)
    
    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        
        state = env.reset()
    
        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            if done: 
                # para estados terminais
                V_next_state = 0
                next_state = env.reset()
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            model[state,action] = (reward, next_state, done)

            # indirect RL / planejamento
            planning(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state        
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 1000 passos, imprime informação sobre o progresso 
        if VERBOSE and ((i+1) % 1000 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q

In [ ]:
EPISODES = 500
LR = 0.3
GAMMA = 0.90
EPSILON = 0.1

rmax = 10.0

rewards, qtable = run_dyna_q(env, EPISODES, LR, GAMMA, EPSILON, 2)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

In [ ]:
# Mostra um gráfico de passos x retornos não descontados acumulados
plot_result(rewards, rmax, cumulative=False, window=10)

## 3 - Experimentos com "Taxi-v3"

### 3.1 - Otimizando Parâmetros do Q-Learning

In [ ]:
RUNS_PER_TRIAL = 3
EPISODES_PER_TRIAL = 200
ENV = gym.make("Taxi-v3")

In [ ]:
def train(trial : optuna.Trial):
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('lr', 0.1, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    gamma = trial.suggest_uniform('gamma', 0.5, 1.0)

    print(f"\nTRIAL #{trial.number}: lr={lr}, eps={eps}, gamma={gamma}")

    # roda o algoritmo várias vezes
    results = repeated_exec(RUNS_PER_TRIAL, "qlearn-optuna", run_qlearning, ENV, EPISODES_PER_TRIAL, lr=lr, epsilon=eps, gamma=gamma)
    
    # soma dos retornos não-descontado finais (dos últimos 20 episódios)
    #return np.sum(results[1][:,-1])
    return np.sum(results[1][:,-20:])

In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_planning.db',
                        study_name=f'qlearning_taxi',
                        load_if_exists=True)

study.optimize(train, n_trials=100)
clear_output()

print("MELHORES PARÂMETROS:")
print(study.best_params)
qlearn_params_taxi = study.best_params

### 3.2 - Experimentos Comparativos

In [ ]:
environment = gym.make("Taxi-v3")
EPISODES = 1_000
RUNS = 20

results = []

results.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_taxi) )
clear_output()

results.append( repeated_exec(RUNS, f"Dyna-Q (1 passo)", run_dyna_q, environment, EPISODES, **qlearn_params_taxi, planning_steps=1) )
clear_output()

results.append( repeated_exec(RUNS, f"Dyna-Q (5 passos)", run_dyna_q, environment, EPISODES, **qlearn_params_taxi, planning_steps=5) )
clear_output()


In [ ]:
plot_multiple_results(results, cumulative='avg', x_log_scale=False, window=1)

In [ ]:
plot_multiple_results(results, cumulative='no', x_log_scale=True, window=1)

## 4 - Experimentos com "FrozenLake"

### 4.1 - Otimizando Parâmetro do Q-Learning

In [ ]:
RUNS_PER_TRIAL = 7
EPISODES_PER_TRIAL = 1000
ENV = gym.make("FrozenLake-v1")

In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///optuna_planning.db',
                        study_name=f'qlearning_frozen_',
                        load_if_exists=True)

study.optimize(train, n_trials=50)
clear_output()

print("MELHORES PARÂMETROS:")
print(study.best_params)
qlearn_params_frozen = study.best_params

### 4.2 - Experimentos Comparativos

In [ ]:
environment = gym.make("FrozenLake-v1")
EPISODES = 7_000
RUNS = 20

results = []

results.append( repeated_exec(RUNS, f"Q-Learning ", run_qlearning, environment, EPISODES, **qlearn_params_frozen) )
clear_output()

results.append( repeated_exec(RUNS, f"Dyna-Q (5 passos)", run_dyna_q, environment, EPISODES, **qlearn_params_frozen, planning_steps=5) )
clear_output()

In [ ]:
plot_multiple_results(results, cumulative='avg', x_log_scale=False, window=1)

### 4.3 - Desafio

Parece que o Dyna-Q não foi tão bem no ambiente `Taxi-v3`...

**Por quê?**

Proponha uma modificação simples no modelo do Dyna-Q para melhorar o desempenho dele nesse ambiente.

Depois, refaça os experimentos da seção 4.2 para conferir se conseguiu. 

In [ ]:
def update_model_new(model, state, action, reward, next_state, is_terminal):
    model[state,action,next_state] = (reward, is_terminal)

def planning_new(model, planning_steps, Q, lr, gamma):
    all_s_a_next = list(model.keys())
    if len(all_s_a_next) < planning_steps:
        samples = rand.choices(all_s_a_next, k=planning_steps)
    else:
        samples = rand.sample(all_s_a_next, k=planning_steps)
    
    for s, a, next_s in samples:
        r, is_terminal = model[(s,a,next_s)]
        if is_terminal:
            V_next_s = 0
        else:
            V_next_s = np.max(Q[next_s]) 
        delta = (r + gamma * V_next_s) - Q[s,a]
        Q[s,a] = Q[s,a] + lr * delta

In [ ]:
# Algoritmo Dyna Q
def run_dyna_q_new(env, episodes, lr=0.1, gamma=0.95, epsilon=0.1, planning_steps=5):
    assert isinstance(env.observation_space, gym.spaces.Discrete)
    assert isinstance(env.action_space, gym.spaces.Discrete)

    num_actions = env.action_space.n

   # inicializa a tabela Q
    Q = np.zeros(shape=(env.observation_space.n, num_actions))

    model = dict({})  # keys are pairs (state, action)
    
    # para cada episódio, guarda sua soma de recompensas (retorno não-descontado)
    sum_rewards_per_ep = []

    # loop principal
    for i in range(episodes):
        done = False
        sum_rewards, reward = 0, 0
        
        state = env.reset()
    
        # executa 1 episódio completo, fazendo atualizações na Q-table
        while not done:
                
            # escolhe a próxima ação -- usa epsilon-greedy
            action = epsilon_greedy(Q, state, num_actions, epsilon)
        
            # realiza a ação, ou seja, dá um passo no ambiente
            next_state, reward, done, _ = env.step(action)
            
            if done: 
                # para estados terminais
                V_next_state = 0
                next_state = env.reset()
            else:
                # para estados não-terminais -- valor máximo (melhor ação)
                V_next_state = np.max(Q[next_state])

            # atualiza a Q-table / direct RL
            delta = (reward + gamma * V_next_state) - Q[state,action]
            Q[state,action] = Q[state,action] + lr * delta

            # atualiza o modelo
            update_model_new(model, state, action, reward, next_state, done)

            # indirect RL / planejamento
            planning_new(model, planning_steps, Q, lr, gamma)

            sum_rewards += reward
            state = next_state        
        
        sum_rewards_per_ep.append(sum_rewards)

        # a cada 1000 passos, imprime informação sobre o progresso 
        if VERBOSE and ((i+1) % 1000 == 0):
            avg_reward = np.mean(sum_rewards_per_ep[-100:])
            print(f"Step {i+1} Average Reward (last 100): {avg_reward:.3f}")

    state = env.reset()
    reward = 0

    return sum_rewards_per_ep, Q

In [ ]:
results.append( repeated_exec(RUNS, f"Dyna-Q New (5 passos)", run_dyna_q_new, environment, EPISODES, **qlearn_params_frozen, planning_steps=5) )
clear_output()

In [ ]:
plot_multiple_results(results, cumulative='avg', x_log_scale=False, window=1)